In [1]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.stats.mstats import winsorize

from Constants import Constants as const

In [10]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20240825_stock_act_reg_data.dta'))
new_year_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_shock_year_data_compustat.csv'),
                                     usecols=[const.GVKEY, 'year', 'post', 'freq']).rename(
    columns={'post': 'post60min', 'freq': 'freq60min', 'year': 'fiscal_year'})
# new_quarterly_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_year_quarter_data.csv'))

In [14]:
reg_df_60: DataFrame = reg_df.merge(new_year_df, on=[const.GVKEY, const.YEAR], how='left').drop(['busdesc'], axis=1)
event_index = reg_df_60.loc[reg_df_60[const.YEAR].apply(lambda x: 2008 < x < 2015)].index
reg_df_60.loc[event_index, 'freq60min'] = reg_df_60['freq60min'].fillna(0)
reg_df_60.loc[event_index, 'ln_freq60min'] = reg_df_60['freq60min'].apply(lambda x: np.log(x + 1))
reg_df_60.loc[event_index, 'freq60min'] = (reg_df_60.loc[event_index, const.YEAR] > 2011).astype(int)
reg_df_60.to_stata(os.path.join(const.RESULT_PATH, '20240918_stock_act_reg_data.dta'),
                   write_index=False)

In [20]:
# merge quarterly data
new_quarterly_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_year_quarter_data_gvkey.csv'))
majgovcustomer_gvkey = reg_df.loc[reg_df['MajorGovCustomer'] == 1, 'gvkey'].unique()
new_quarterly_df.loc[:, 'MajorGovCustomer'] = 0
new_quarterly_df.loc[new_quarterly_df[const.GVKEY].isin(majgovcustomer_gvkey), 'MajorGovCustomer'] = 1

new_quarterly_df['yearquarter'] = new_quarterly_df.apply(lambda x: '{}q{}'.format(int(x['Year']), int(x['Quarter'])), axis=1)
new_quarterly_df.loc[:, 'post'] = (new_quarterly_df['Year'] > 2011).astype(int)
new_quarterly_df['ln_freq'] = new_quarterly_df['freq'].apply(lambda x: np.log(x + 1))

int_terms = list()
for year in range(2008, 2015):
    for quarter in range(1, 5):
        new_quarterly_df.loc[:, f'd{year}q{quarter}'] = (new_quarterly_df['yearquarter'] == f'{year}q{quarter}').astype(int)
        int_terms.append(f'1.d{year}q{quarter}#1.MajorGovCustomer')

new_quarterly_df.to_stata(os.path.join(const.RESULT_PATH, '20240918_stock_act_reg_data_quarterly.dta'), write_index=False)

print(' '.join(int_terms))

1.d2008q1#1.MajorGovCustomer 1.d2008q2#1.MajorGovCustomer 1.d2008q3#1.MajorGovCustomer 1.d2008q4#1.MajorGovCustomer 1.d2009q1#1.MajorGovCustomer 1.d2009q2#1.MajorGovCustomer 1.d2009q3#1.MajorGovCustomer 1.d2009q4#1.MajorGovCustomer 1.d2010q1#1.MajorGovCustomer 1.d2010q2#1.MajorGovCustomer 1.d2010q3#1.MajorGovCustomer 1.d2010q4#1.MajorGovCustomer 1.d2011q1#1.MajorGovCustomer 1.d2011q2#1.MajorGovCustomer 1.d2011q3#1.MajorGovCustomer 1.d2011q4#1.MajorGovCustomer 1.d2012q1#1.MajorGovCustomer 1.d2012q2#1.MajorGovCustomer 1.d2012q3#1.MajorGovCustomer 1.d2012q4#1.MajorGovCustomer 1.d2013q1#1.MajorGovCustomer 1.d2013q2#1.MajorGovCustomer 1.d2013q3#1.MajorGovCustomer 1.d2013q4#1.MajorGovCustomer 1.d2014q1#1.MajorGovCustomer 1.d2014q2#1.MajorGovCustomer 1.d2014q3#1.MajorGovCustomer 1.d2014q4#1.MajorGovCustomer


In [24]:
inter_terms = '1.d2009q1#1.MajorGovCustomer 1.d2009q2#1.MajorGovCustomer 1.d2009q3#1.MajorGovCustomer 1.d2009q4#1.MajorGovCustomer 1.d2010q1#1.MajorGovCustomer 1.d2010q2#1.MajorGovCustomer 1.d2010q3#1.MajorGovCustomer 1.d2010q4#1.MajorGovCustomer 1.d2011q1#1.MajorGovCustomer 1.d2011q2#1.MajorGovCustomer 1.d2011q3#1.MajorGovCustomer 1.d2011q4#1.MajorGovCustomer 1.d2012q1#1.MajorGovCustomer 1.d2012q2#1.MajorGovCustomer 1.d2012q3#1.MajorGovCustomer 1.d2012q4#1.MajorGovCustomer 1.d2013q1#1.MajorGovCustomer 1.d2013q2#1.MajorGovCustomer 1.d2013q3#1.MajorGovCustomer 1.d2013q4#1.MajorGovCustomer 1.d2014q1#1.MajorGovCustomer 1.d2014q2#1.MajorGovCustomer 1.d2014q3#1.MajorGovCustomer 1.d2014q4#1.MajorGovCustomer'.split(' ')
xlabel = list()
for i, term in enumerate(inter_terms):
    year = term.split('q')[0][-4:]
    quarter = term.split('q')[1][0]
    xlabel.append(f'{i+1} "{year}q{quarter}"')
    
print(' '.join(xlabel))

1 "2009q1" 2 "2009q2" 3 "2009q3" 4 "2009q4" 5 "2010q1" 6 "2010q2" 7 "2010q3" 8 "2010q4" 9 "2011q1" 10 "2011q2" 11 "2011q3" 12 "2011q4" 13 "2012q1" 14 "2012q2" 15 "2012q3" 16 "2012q4" 17 "2013q1" 18 "2013q2" 19 "2013q3" 20 "2013q4" 21 "2014q1" 22 "2014q2" 23 "2014q3" 24 "2014q4"


# Merge Analysts FCSTERROR DISPERSION market information data.

In [26]:
# load regression data
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241009_stock_act_reg_data_v2.dta'))

# load sue123 data
sue123_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241006_sue123_data_annual.pkl'))

# load analysts fcsterror dispersion data
adf_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241006_analysts_dispersion_fcsterror.pkl'))

# load synchrony data
synchrony_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241006_synchrony_weekly.pkl'))
synchrony_df_monthly: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241010_synchrony_monthly.pkl'))
synchrony_df_daily: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241010_synchrony_daily.pkl'))
synchrony_df: DataFrame = synchrony_df.merge(synchrony_df_monthly, on=['PERMNO', 'year'], how='left', suffixes=('', '_MONTH')).merge(
    synchrony_df_daily, on=['PERMNO', 'year'], how='left')

In [35]:
# merge with sue 123 data
sue123_df['gvkey'] = sue123_df['gvkey'].astype(int)
sue123_df['fyearq'] = sue123_df['fyearq'].astype(int)
sue123_df.rename(columns={'fyearq': const.YEAR}, inplace=True)

reg_df2: DataFrame = reg_df.merge(sue123_df, on=[const.GVKEY, const.YEAR], how='left')

In [23]:
adf_df[adf_df[['ticker', 'fyear']].duplicated()]

,ticker,fpedats,anndats_act,meanest,numest,stdev,actual,prcc_f,fyear,ANALYSTS,lnANALYSTS,DISPERSION,FCSTERROR,fiscal_year
9,EXPD,2013-12-31,2014-02-25,1.96,1.0,NaN,1.68,44.25,2013.0,1.0,0.693147,NaN,0.006328,2013.0
10,JJSF,2010-03-31,2010-04-22,0.46,3.0,0.03,0.48,43.19,2009.0,3.0,1.386294,0.000695,0.000463,2009.0
11,NSM,2009-05-31,2009-06-11,-0.42,7.0,0.02,-0.28,NaN,2008.0,7.0,2.079442,NaN,NaN,2008.0
25,NSM,2010-08-31,2010-09-09,0.21,14.0,0.03,0.36,NaN,2009.0,14.0,2.708050,NaN,NaN,2009.0
27,WGL,2016-06-30,2016-08-03,0.11,4.0,0.19,0.33,57.67,2015.0,4.0,1.609438,0.003295,0.003815,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155663,FOSL,2006-09-30,2006-11-14,0.34,9.0,0.02,0.32,21.51,2005.0,9.0,2.302585,0.000930,0.000930,2005.0
155664,LF,2010-03-31,2010-05-03,-0.29,5.0,0.04,-0.37,3.91,2009.0,5.0,1.791759,0.010230,0.020460,2009.0
155665,OXPS,2006-12-31,2007-01-31,0.95,4.0,0.06,1.15,22.69,2006.0,4.0,1.609438,0.002644,0.008814,2006.0
155666,STGN,2006-12-31,2007-03-06,0.40,2.0,0.03,0.25,7.44,2006.0,2.0,1.098612,0.004032,0.020161,2006.0


In [36]:
# merge adf data
# adf_df.rename(columns={'tic': 'ticker'}, inplace=True)
adf_df.loc[:, const.YEAR] = adf_df['fyear']
adf_valid: DataFrame = adf_df.loc[:, ['ticker', const.YEAR, 'ANALYSTS', 'lnANALYSTS', 'DISPERSION', 'FCSTERROR']]

reg_df3: DataFrame = reg_df2.merge(adf_valid, on=['ticker', const.YEAR], how='left')
reg_df3.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,QFM_f_annual,predicted_costcap,predicted_hurdle,annual_sue1,annual_sue2,annual_sue3,ANALYSTS,lnANALYSTS,DISPERSION,FCSTERROR
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,11630.000000,8505.000000,8505.000000,6726.000000,6726.000000,6032.000000,3948.000000,3948.000000,3.778000e+03,3828.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,2.679648,0.092023,0.117034,0.000310,0.000311,-0.032550,11.545593,2.289246,5.529143e+02,40.643852
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,-0.879652,0.049550,0.060928,-0.580058,-0.580058,-6.885936,1.000000,0.693147,0.000000e+00,0.000000
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,0.207093,0.083509,0.105448,0.000000,0.000000,-0.364511,5.000000,1.791759,1.275257e-03,0.003183
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,0.434483,0.092779,0.118525,0.000000,0.000000,0.000000,9.000000,2.302585,3.047777e-03,0.008994
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,0.895383,0.101097,0.129915,0.000000,0.000000,0.365300,16.000000,2.833213,9.602861e-03,0.029951
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,4931.937993,0.124678,0.167867,1.412761,1.412761,7.976155,56.000000,4.043051,1.899558e+06,134482.758621
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,59.815111,0.012435,0.017691,0.023666,0.023675,0.970669,8.677086,0.719222,3.102871e+04,2178.351333


In [37]:
# Define the columns to winsorize
columns_to_winsorize = ['DISPERSION', 'FCSTERROR']

# Winsorize each column in the dataset reg_df3 at the 1st and 99th percentiles
for column in columns_to_winsorize:
    non_na_data = reg_df3[column].dropna()
    winsorized_data = winsorize(non_na_data, limits=[0.01, 0.01])
    reg_df3.loc[non_na_data.index, column] = winsorized_data

# Display the updated dataframe
reg_df3.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,QFM_f_annual,predicted_costcap,predicted_hurdle,annual_sue1,annual_sue2,annual_sue3,ANALYSTS,lnANALYSTS,DISPERSION,FCSTERROR
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,11630.000000,8505.000000,8505.000000,6726.000000,6726.000000,6032.000000,3948.000000,3948.000000,3778.000000,3828.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,2.679648,0.092023,0.117034,0.000310,0.000311,-0.032550,11.545593,2.289246,0.239984,0.357989
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,-0.879652,0.049550,0.060928,-0.580058,-0.580058,-6.885936,1.000000,0.693147,0.000100,0.000244
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,0.207093,0.083509,0.105448,0.000000,0.000000,-0.364511,5.000000,1.791759,0.001275,0.003183
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,0.434483,0.092779,0.118525,0.000000,0.000000,0.000000,9.000000,2.302585,0.003048,0.008994
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,0.895383,0.101097,0.129915,0.000000,0.000000,0.365300,16.000000,2.833213,0.009603,0.029951
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,4931.937993,0.124678,0.167867,1.412761,1.412761,7.976155,56.000000,4.043051,15.540258,20.199805
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,59.815111,0.012435,0.017691,0.023666,0.023675,0.970669,8.677086,0.719222,1.672322,2.191517


In [27]:
# merge sync data
synchrony_df.rename(columns={'TICKER': 'ticker', 'year': const.YEAR}, inplace=True)
crsp_comp_link: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, 'crsp_comp.pkl')).rename(columns={'permco': 'PERMNO'})
synchrony_df_gvkey = synchrony_df.merge(crsp_comp_link, on=['PERMNO'], how='left')
synchrony_df_gvkey['gvkey'] = pd.to_numeric(synchrony_df_gvkey['gvkey'], errors='coerce')

In [29]:
synchrony_df_gvkey.keys()

Index(['PERMNO', 'fiscal_year', 'SYNCHRONICITY', 'SYNCHRONICITY_MKT',
       'SYNCHRONICITY_IND', 'ticker', 'SYNCHRONICITY_MONTH',
       'SYNCHRONICITY_MKT_MONTH', 'SYNCHRONICITY_IND_MONTH', 'SYNCHRONICITY_D',
       'SYNCHRONICITY_MKT_D', 'SYNCHRONICITY_IND_D', 'gvkey'],
      dtype='object')

In [30]:
reg_df2: DataFrame = reg_df.drop(['SYNCHRONICITY', 'SYNCHRONICITY_MKT', 'SYNCHRONICITY_IND'], axis=1).merge(
    synchrony_df_gvkey.drop(['ticker', 'PERMNO'], axis=1).dropna(subset=['gvkey']), on=['gvkey', const.YEAR], how='left', suffixes=('', '_gvkey')).merge(
    synchrony_df_gvkey.drop(['gvkey', 'PERMNO'], axis=1).dropna(subset=['ticker']), on=['ticker', const.YEAR], how='left', suffixes=('', '_ticker')).merge(
    synchrony_df_gvkey.drop(['gvkey', 'ticker'], axis=1).dropna(subset=['PERMNO']), on=['PERMNO', const.YEAR], how='left', suffixes=('', '_PERMNO'))

key_to_drop = list()
for key in ['SYNCHRONICITY', 'SYNCHRONICITY_MKT', 'SYNCHRONICITY_IND', 'SYNCHRONICITY_MONTH','SYNCHRONICITY_MKT_MONTH', 
            'SYNCHRONICITY_IND_MONTH', 'SYNCHRONICITY_D', 'SYNCHRONICITY_MKT_D', 'SYNCHRONICITY_IND_D']:
    reg_df2[key] = reg_df2[key].fillna(reg_df2[f'{key}_ticker']).fillna(reg_df2[f'{key}_PERMNO'])
    key_to_drop.extend([f'{key}_ticker', f'{key}_PERMNO'])
    
reg_df2.drop(key_to_drop, axis=1, inplace=True)
reg_df2.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,numest,SYNCHRONICITY,SYNCHRONICITY_MKT,SYNCHRONICITY_IND,SYNCHRONICITY_MONTH,SYNCHRONICITY_MKT_MONTH,SYNCHRONICITY_IND_MONTH,SYNCHRONICITY_D,SYNCHRONICITY_MKT_D,SYNCHRONICITY_IND_D
count,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,12671.000000,...,7298.000000,7702.000000,7716.000000,7716.000000,7576.000000,7629.000000,7629.000000,7735.000000,7737.000000,7737.000000
mean,5917.137085,78643.466183,2011.542578,0.350485,4.630258,0.836249,0.147818,0.508168,6.555644,0.180124,...,102.521102,-0.646085,-1.035348,-2.472007,1.479553,-0.077860,-0.460027,-0.983703,-1.249030,-1.325857
min,0.000000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,1.000000,-4.990253,-7.275702,-10.594919,-2.934787,-7.198360,-7.505549,-5.942492,-7.276499,-8.105904
25%,3026.000000,12215.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.081295,0.000892,...,38.000000,-1.263271,-1.696400,-3.115373,0.644250,-0.774217,-1.211535,-1.721788,-1.909017,-2.112200
50%,5773.000000,61587.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.608994,0.148389,...,80.000000,-0.614289,-0.938245,-2.354456,1.387043,-0.015227,-0.387299,-0.837935,-0.981284,-1.116047
75%,8931.500000,150699.000000,2014.000000,1.000000,6.000000,1.945910,0.000000,1.000000,7.969589,0.267840,...,151.000000,0.028338,-0.211519,-1.750940,2.228142,0.742137,0.386116,-0.078576,-0.316189,-0.301435
max,12073.000000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,571.000000,6.829389,3.986925,3.496632,36.043653,10.738283,15.020359,2.880722,1.724813,2.586106
std,3461.817696,73399.399565,2.280434,0.477141,8.419172,1.232130,0.354933,0.499953,2.131880,0.269232,...,83.724776,0.963188,1.157417,1.095382,1.581823,1.228575,1.285444,1.248980,1.326810,1.427007


In [14]:
reg_df2.keys()

Index(['index', 'gvkey', 'fiscal_year', 'mf_indicator', 'frequency',
       'log_frequency', 'MajorGovCustomer', 'post', 'log_market_value', 'lev',
       ...
       'DISPERSION', 'FCSTERROR', 'SYNCHRONICITY', 'SYNCHRONICITY_MKT',
       'SYNCHRONICITY_IND', 'PERMNO', 'cpie_gpin', 'cpie_owr', 'coverage',
       'numest'],
      dtype='object', length=191)

In [31]:
columns_to_winsorize = ['DISPERSION', 'FCSTERROR', 'SYNCHRONICITY', 'SYNCHRONICITY_MKT',
       'SYNCHRONICITY_IND', 'cpie_gpin', 'cpie_owr', 'coverage',
       'numest', 'SYNCHRONICITY_MONTH','SYNCHRONICITY_MKT_MONTH', 'SYNCHRONICITY_IND_MONTH',
                        'SYNCHRONICITY_D', 'SYNCHRONICITY_MKT_D', 'SYNCHRONICITY_IND_D']
reg_df3: DataFrame = reg_df2.drop_duplicates([const.GVKEY, const.YEAR], keep='first')
# Winsorize each column in the dataset reg_df3 at the 1st and 99th percentiles
for column in columns_to_winsorize:
    non_na_data = reg_df3[column].dropna()
    winsorized_data = winsorize(non_na_data, limits=[0.01, 0.01])
    reg_df3.loc[non_na_data.index, column] = winsorized_data

# Display the updated dataframe
reg_df3.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,numest,SYNCHRONICITY,SYNCHRONICITY_MKT,SYNCHRONICITY_IND,SYNCHRONICITY_MONTH,SYNCHRONICITY_MKT_MONTH,SYNCHRONICITY_IND_MONTH,SYNCHRONICITY_D,SYNCHRONICITY_MKT_D,SYNCHRONICITY_IND_D
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,6706.000000,7109.000000,7122.000000,7122.000000,6992.000000,7037.000000,7037.000000,7138.000000,7140.000000,7140.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,98.629884,-0.670551,-1.064322,-2.490808,1.435795,-0.108291,-0.464465,-1.015247,-1.293022,-1.361813
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,3.000000,-3.101548,-4.330298,-5.564165,-1.286456,-3.487743,-3.837932,-4.285619,-5.212167,-5.379391
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,36.000000,-1.308171,-1.765061,-3.146918,0.615697,-0.839259,-1.209620,-1.799696,-1.999417,-2.196078
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,72.000000,-0.634087,-0.961595,-2.378148,1.386042,-0.036902,-0.417715,-0.863187,-1.057522,-1.174833
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,137.750000,0.006029,-0.238802,-1.750827,2.193494,0.714125,0.358543,-0.100233,-0.339558,-0.325174
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,374.000000,1.319524,1.131258,-0.237203,4.726006,2.596473,2.385805,1.394683,0.957713,1.317516
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,83.429290,0.945929,1.139099,1.069094,1.200927,1.202594,1.213881,1.260805,1.329277,1.436088


In [32]:
reg_df3.to_stata(os.path.join(const.RESULT_PATH, '20241010_stock_act_reg_data_v3.dta'), write_index=False, version=117)

In [31]:
import zipfile

# Define the path to the zip file
zip_file_path = os.path.join(const.DATA_PATH, 'cpie_data.zip')
csv_file_name = 'cpie_daily.csv'

# Open the zip file and read the CSV file
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # Extract the CSV file from the zip and load it into a pandas DataFrame
    with z.open(csv_file_name) as f:
        df = pd.read_csv(f)

cpin_daily = df.loc[:, ['permno', 'year', 'cpie_gpin', 'cpie_owr', 'date']].copy()

In [33]:
cpin_daily_1316 = pd.read_pickle(os.path.join(const.TEMP_PATH, '2013_2016_gpin_owr.pkl')).loc[:, ['date', 'permno', 'year', 'gpin', 'owr']].rename(
    columns={'gpin': 'cpie_gpin', 'owr': 'cpie_owr'})

cpin_daily_full = pd.concat([cpin_daily, cpin_daily_1316], axis=0, ignore_index=True)

cpin_daily_full['date'] = pd.to_datetime(cpin_daily_full['date'])

# Display the first few rows of the DataFrame to ensure it loaded correctly
print(cpin_daily_full.head())

   permno  year  cpie_gpin  cpie_owr       date
0   10057  1993   0.010702  0.848057 1993-01-04
1   10057  1993   0.070915  0.875569 1993-01-05
2   10057  1993   0.013788  0.911562 1993-01-06
3   10057  1993   0.432916  0.815852 1993-01-07
4   10057  1993   0.002560  0.724871 1993-01-08


In [36]:
cpin_daily_full['fqtr'] = cpin_daily_full['date'].dt.to_period('Q')

In [7]:
annual_gpin_owr = cpin_daily_full.groupby(['permno', 'year']).agg({
    'cpie_gpin': 'mean',
    'cpie_owr': 'mean',
}).reset_index(drop=False)
annual_gpin_owr.rename(columns={'year': const.YEAR}, inplace=True)

In [37]:
quarter_gpin_owr = cpin_daily_full.groupby(['permno', 'fqtr']).agg({
    'cpie_gpin': 'mean',
    'cpie_owr': 'mean',
}).reset_index(drop=False)

In [8]:
crsp_comp_link: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, 'crsp_comp.pkl'))
annual_gpin_owr_link = annual_gpin_owr.merge(crsp_comp_link, left_on='permno', right_on='permco', how='left')

In [9]:
annual_gpin_owr_link['gvkey'] = pd.to_numeric(annual_gpin_owr_link['gvkey'], errors='coerce')

In [40]:
quarter_gpin_owr_link = quarter_gpin_owr.merge(crsp_comp_link.rename(columns={"permco": 'permno'}), left_on='permno', right_on='permno', how='left')
quarter_gpin_owr_link['gvkey'] = pd.to_numeric(quarter_gpin_owr_link['gvkey'], errors='coerce')
quarter_gpin_owr_link['permno'] = pd.to_numeric(quarter_gpin_owr_link['permno'], errors='coerce')

quarter_gpin_owr_link.to_pickle(os.path.join(const.TEMP_PATH, 'cpin_gpin_owr_quarterly.pkl'))

In [39]:
quarter_gpin_owr_link.head()

,permno,fqtr,cpie_gpin,cpie_owr,gvkey
0,10001,2014Q1,0.421506,0.543786,NaN
1,10001,2014Q2,0.481146,0.557784,NaN
2,10001,2014Q3,0.337435,0.530663,NaN
3,10001,2014Q4,0.365699,0.514144,NaN
4,10001,2015Q1,0.325230,0.413716,NaN


In [25]:
reg_df4: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241011_stock_act_reg_data_v2.dta'))

reg_df5: DataFrame = reg_df4.merge(
    annual_gpin_owr_link.drop(['permno', 'permco'], axis=1).dropna(subset=['gvkey']), on=[const.GVKEY, const.YEAR], 
    how='left', suffixes=("", "_gvkey")).merge(
    annual_gpin_owr_link.drop(['gvkey', 'permco'], axis=1).dropna(subset=['permno']), left_on=['PERMNO', const.YEAR], 
    right_on=['permno', const.YEAR], how='left', suffixes=('', '_permno')).merge(
    annual_gpin_owr_link.drop(['gvkey', 'permno'], axis=1), left_on=['PERMNO', const.YEAR], 
    right_on=['permco', const.YEAR], how='left', suffixes=('', '_permco'))

key_to_drop = ['permco', 'permno']
for key in ['cpie_gpin', 'cpie_owr']:
    reg_df5[key] = reg_df5[key].fillna(reg_df5[f'{key}_gvkey']).fillna(reg_df5[f'{key}_permno']).fillna(reg_df5[f'{key}_permco'])
    key_to_drop.append(f'{key}_permno')
    key_to_drop.append(f'{key}_permco')
    key_to_drop.append(f'{key}_gvkey')

reg_df5.drop(key_to_drop, axis=1, inplace=True)

In [28]:
reg_df5.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,SYNCHRONICITY_MKT_D,SYNCHRONICITY_IND_D,numest,numest_last,DISPERSION,DISPERSION_last,FCSTERROR,FCSTERROR_last,permno,permco
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,7140.000000,7140.000000,6637.000000,6637.000000,6219.000000,6219.000000,6570.000000,6570.000000,2754.000000,256.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,-1.293022,-1.361813,8.781616,8.777158,0.084595,0.057665,0.146156,0.118434,61329.095861,36007.843750
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,-5.212167,-5.379391,1.000000,1.000000,0.000094,0.000000,0.000035,0.000000,10104.000000,10104.000000
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,-1.999417,-2.196078,3.166667,3.000000,0.001137,0.000652,0.001680,0.000920,30940.000000,15684.750000
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,-1.057522,-1.174833,6.333333,6.000000,0.002691,0.001626,0.005238,0.003289,75735.500000,41080.000000
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,-0.339558,-0.325174,12.583333,13.000000,0.007994,0.005217,0.018693,0.012740,86799.000000,54704.000000
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,0.957713,1.317516,54.666667,54.000000,4.209722,3.080000,7.087390,5.981308,93436.000000,59619.000000
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,1.329277,1.436088,7.529601,7.763877,0.475061,0.341575,0.792531,0.676268,28727.498355,19106.167438


In [27]:
reg_df5.drop_duplicates(subset=[const.GVKEY, const.YEAR], keep='first', inplace=True)

In [29]:
reg_df5.to_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v1.dta'),
                 write_index=False, version=117)

In [21]:
[i for i in reg_df5.keys() if i.startswith('ticker')]

['ticker']

In [20]:
annual_analyst_df: DataFrame = pd.read_csv(os.path.join(const.TEMP_PATH, '2007_2016_annual_analyst_coverage.csv')).rename(columns={'year': const.YEAR})
df_df: DataFrame = pd.read_csv(os.path.join(const.TEMP_PATH, '2007_2016_annual_dispersion_fcsterror.csv'), 
                               usecols=['ticker', 'year', 'numest', 'permno', 'DISPERSION', 'FCSTERROR', 'numest_last'])
df_df_gvkey: DataFrame = df_df.merge(crsp_comp_link, left_on='permno', right_on='permco', how='left').rename(columns={'year': const.YEAR})
df_df_gvkey['gvkey'] = pd.to_numeric(df_df_gvkey['gvkey'], errors='coerce')

In [24]:
df_df_gvkey.keys()

Index(['ticker', 'fiscal_year', 'numest', 'permno', 'DISPERSION', 'FCSTERROR',
       'permco', 'gvkey'],
      dtype='object')

In [29]:
reg_df6: DataFrame = reg_df5.merge(annual_analyst_df.drop(['ticker', 'permno'], axis=1).dropna(subset=[const.GVKEY]), on=[const.GVKEY, const.YEAR], how='left').merge(
    annual_analyst_df.drop(['gvkey', 'ticker'], axis=1).dropna(subset=['permno']), left_on=['PERMNO', const.YEAR], right_on=['permno', const.YEAR], how='left', suffixes=('', '_permno')).merge(annual_analyst_df.drop(['gvkey', 'permno'], axis=1).dropna(subset=['ticker']), left_on=['ticker', const.YEAR], right_on=['ticker', const.YEAR], how='left', suffixes=('', '_ticker'))

key_to_drop = list()
for key in ['coverage']:
    reg_df6[key] = reg_df6[key].fillna(reg_df6[f'{key}_permno']).fillna(reg_df6[f'{key}_ticker'])
    key_to_drop.append(f'{key}_permno')
    key_to_drop.append(f'{key}_ticker')


In [32]:
reg_df6.drop(['permno', 'permco'], axis=1, inplace=True)

In [41]:
reg_df7: DataFrame = reg_df6.merge(df_df_gvkey.drop(['ticker', 'permno', 'permco'], axis=1).dropna(subset=[const.GVKEY]), on=[const.GVKEY, const.YEAR], how='left', suffixes=("", "_gvkey")).merge(
    df_df_gvkey.drop(['gvkey', 'ticker', 'permco'], axis=1).dropna(subset=['permno']), left_on=['PERMNO', const.YEAR], right_on=['permno', const.YEAR], how='left', suffixes=('', '_permno')).merge(
    df_df_gvkey.drop(['gvkey', 'permno', 'permco'], axis=1).dropna(subset=['ticker']), left_on=['ticker', const.YEAR], right_on=['ticker', const.YEAR], how='left', suffixes=('', '_ticker')).merge(
    df_df_gvkey.drop(['gvkey', 'permno', 'ticker'], axis=1).dropna(subset=['permco']), left_on=['PERMNO', const.YEAR], right_on=['permco', const.YEAR], how='left', suffixes=('', '_permco'))

for key in ['numest', 'DISPERSION', 'FCSTERROR']:
    if key != 'numest':
        reg_df7[key] = reg_df7[f'{key}_gvkey'].fillna(reg_df7[key])
        key_to_drop.append(f'{key}_gvkey')
    reg_df7[key] = reg_df7[key].fillna(reg_df7[f'{key}_permno']).fillna(reg_df7[f'{key}_ticker']).fillna(reg_df7[f'{key}_permco'])
    key_to_drop.append(f'{key}_permno')
    key_to_drop.append(f'{key}_ticker')
    key_to_drop.append(f'{key}_permco')

reg_df7['PERMNO'] = reg_df7['PERMNO'].fillna(reg_df7['permco']).fillna(reg_df7['permno'])
key_to_drop.append('permco')
key_to_drop.append('permno')
key_to_drop.append('permno_permno')
reg_df7.drop(key_to_drop, axis=1, inplace=True)

In [42]:
reg_df7.to_stata(os.path.join(const.RESULT_PATH, '20241009_stock_act_reg_data_v2.dta'), write_index=False, version=117)

In [7]:
# Merge the regression data with FPI = 1 IBES coverage data
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241010_stock_act_reg_data_v3.dta')).drop(
    ['DISPERSION', 'FCSTERROR', 'numest'], axis=1)
crsp_comp_link: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, 'crsp_comp.pkl'))

df_df: DataFrame = pd.read_csv(os.path.join(const.TEMP_PATH, '2007_2016_annual_dispersion_fcsterror.csv'), 
                               usecols=['ticker', 'year', 'numest', 'permno', 'DISPERSION', 'FCSTERROR', 'numest_last', 'DISPERSION_last', 'FCSTERROR_last'])
df_df_gvkey: DataFrame = df_df.merge(crsp_comp_link, left_on='permno', right_on='permco', how='left').rename(columns={'year': const.YEAR})
df_df_gvkey['gvkey'] = pd.to_numeric(df_df_gvkey['gvkey'], errors='coerce')

reg_df2: DataFrame = reg_df.merge(df_df_gvkey.drop(['ticker', 'permno', 'permco'], axis=1).dropna(subset=[const.GVKEY]), on=[const.GVKEY, const.YEAR], how='left').merge(
    df_df_gvkey.drop(['gvkey', 'ticker', 'permco'], axis=1).dropna(subset=['permno']), left_on=['PERMNO', const.YEAR], right_on=['permno', const.YEAR], how='left', suffixes=('', '_permno')).merge(
    df_df_gvkey.drop(['gvkey', 'permno', 'permco'], axis=1).dropna(subset=['ticker']), left_on=['ticker', const.YEAR], right_on=['ticker', const.YEAR], how='left', suffixes=('', '_ticker')).merge(
    df_df_gvkey.drop(['gvkey', 'permno', 'ticker'], axis=1).dropna(subset=['permco']), left_on=['PERMNO', const.YEAR], right_on=['permco', const.YEAR], how='left', suffixes=('', '_permco'))

key_to_drop = list()
for key in ['numest', 'DISPERSION', 'FCSTERROR', 'numest_last', 'DISPERSION_last', 'FCSTERROR_last']:
    reg_df2[key] = reg_df2[key].fillna(reg_df2[f'{key}_permno']).fillna(reg_df2[f'{key}_ticker']).fillna(reg_df2[f'{key}_permco'])
    key_to_drop.append(f'{key}_permno')
    key_to_drop.append(f'{key}_ticker')
    key_to_drop.append(f'{key}_permco')
    
reg_df2.drop(key_to_drop, axis=1, inplace=True)
reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20241010_stock_act_reg_data_v4.dta'), write_index=False, version=117)

In [2]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241010_stock_act_reg_data_v4.dta'))
reg_df.drop_duplicates([const.GVKEY, const.YEAR], keep='first').to_stata(os.path.join(const.RESULT_PATH, '20241010_stock_act_reg_data_v5.dta'), write_index=False, version=117)